# Project 4


In [1]:
import xgboost as xgb

In [2]:
from xgboost import XGBClassifier

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import pandas as pd
import numpy as np

#must ensure conda install py-xgboost
#import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Loading Data

In [5]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


In [6]:
train_df.head()

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [7]:
test_df.head()

,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17494 entries, 0 to 17493
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_id        17494 non-null  int64  
 1   title            17494 non-null  object 
 2   year             17316 non-null  float64
 3   user_review      17494 non-null  object 
 4   user_suggestion  17494 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 683.5+ KB


In [9]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.user_suggestion.values)


In [10]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.user_review.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

# Starting Models
- TF-IDF
- Logistic Regression

In [11]:
# Following features are common starting points and work well in most cases.
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [12]:
# Fit a simple Logistic Regression on TF-IDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
bin_preds = clf.predict(xvalid_tfv)
# sklearn.metrics.log_loss(y_true, y_pred, *, eps=1e-15, normalize=True, sample_weight=None, labels=None)
from sklearn.metrics import log_loss, f1_score, accuracy_score

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))
#Note, we want to be careful about applying accuracy/f1 here because those are classification metrics, and only apply to the
#binary outputs of .predict.  
#We can apply the predict_proba to get actual probabilities (i.e. before the softmax step) that are possibly more
#descriptive of what the regression is doing.


logloss: 0.384 
f1 score: 0.879 
Accuracy score: 0.858 


Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.

In [13]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [14]:
#checking out an arbitrary entry of that ctv
#xtrain_ctv[1,2]

In [15]:
# Fit a simple Logistic Regression on those Counts
clf = LogisticRegression(C=1.0, max_iter=500,)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
bin_preds = clf.predict(xvalid_ctv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.449 
f1 score: 0.872 
Accuracy score: 0.851 


Ok, worse loss, f1, and accuracy.

In [16]:
# Fitt a simple Naive Bayes on TF-IDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
bin_preds = clf.predict(xvalid_tfv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.400 
f1 score: 0.868 
Accuracy score: 0.836 


Wow!  Naive Bayes improved loss!     

Accuracy and F1 took a hit though, but the improvement in loss implies that the model is overall better at describing things.  

What happens when we use this model on counts data instead?

In [17]:
# Fit a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
bin_preds = clf.predict(xvalid_ctv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 1.286 
f1 score: 0.876 
Accuracy score: 0.859 


NB on counts did worse on loss, but held ground on F1 and Acc.

One more classic algorithm is SVMs. 

In order to improve performance on time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM.

Also, it is better to standardize the data before applying SVMs.

In [18]:
# Apply SVD, with 120 components. 120-200 components are widely considered sufficient for SVM.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

Now time to fit the SVM.  This can take a while!

In [20]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we want probabilities in addition to the 0/1 scores
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
bin_preds = clf.predict(xvalid_svd_scl)


print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.368 
f1 score: 0.858 
Accuracy score: 0.835 


This is even better loss and good f1/acc.

That performed pretty well on logloss.  Neat!  

Of course, we couldn't stop without trying XGBoost!

In [23]:
# Fit a basic xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())
bin_preds = clf.predict(xvalid_tfv.tocsc())


print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.393 
f1 score: 0.856 
Accuracy score: 0.831 


Strangely, XGBoost didn't do as well on TF-IDF features as logistic regression.

In [27]:
# Fitting a simple xgboost on tf-idf svd features with default parameters
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)
bin_preds = clf.predict(xvalid_svd_scl)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:56:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.411 
f1 score: 0.758 
Accuracy score: 0.707 


In [24]:
# Fit a basic xgboost on more common hyperparams for tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)
bin_preds = clf.predict(xvalid_svd)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:51:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.390 
f1 score: 0.847 
Accuracy score: 0.823 


On TF-IDF SVD unscaled features, XGBoost does not improve.  

What about on scaled features?

In [26]:
# Fit a basic xgboost on SCALED tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
bin_preds = clf.predict(xvalid_svd_scl)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:54:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.390 
f1 score: 0.847 
Accuracy score: 0.823 


And on Scaled TF-IDF SVD features, XGboost does very similarly.  

But wait!  We haven't done any tuning of hyperparameters yet.

# Improve all those models with some grid search optimization  
First, we can define a scoring metric for automation.

In [30]:
basic_scorer = metrics.make_scorer(log_loss, greater_is_better=False, needs_proba=True)

Since we applied svd and scaling to two different sets (TF-IDF and SVD), this seems like it has potential for a pipeline approach

In [ ]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize a standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the Logistic Regression pipeline 
clf_lr = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

#Here we can set up another 

In [21]:
#set up apache spark system to do parallel processing?
#maybe Try to set up amazon resources - possibly overkill/not good for github
## see recommendation systems section - this isn't actually directly applicable - more for recs for a user based on other user's recs
# topic 39 - nlp obvs
#tf-idfs?
#nnetworks with
# word2vec (including gensim) see https://learning.flatironschool.com/courses/1890/pages/using-word2vec?module_item_id=261958
#GloVe for classification - (This is a very good pretrained model)
# word embeddings, 
#sequence models e.g. 
#RNN and 
#lstms, 
#grus, 
#gridsearch for best params,
#ensemble methods - e.g. xgboost on these models
#anything else that is in 
# https://www.kaggle.com/code/abhishek/approaching-almost-any-nlp-problem-on-kaggle/comments
'''tfidf
count features
logistic regression
naive bayes
svm
xgboost
grid search
word vectors
LSTM
GRU
Ensembling'''



'tfidf\ncount features\nlogistic regression\nnaive bayes\nsvm\nxgboost\ngrid search\nword vectors\nLSTM\nGRU\nEnsembling'